In [2]:
#### Documents ####
cbt_interviews = 'https://docs.google.com/document/d/1W0kc4UrimQN9YGhKGWsyY59cWyzktALpI9PsWT-m8sY/edit'



In [22]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader

file_path = ['/Users/misanmeggison/Downloads/DepressionGuide-web.pdf', '/Users/misanmeggison/Downloads/AUS_Depression _consumer and carer tx guidelines.pdf']

for file in file_path:
    loader = PyMuPDFLoader(file)
    documents = loader.load()

documents


[Document(page_content='August 2009\nAustralian treatment guide \nfor consumers and carers\nCoping with  \ndepression\n', metadata={'source': '/Users/misanmeggison/Downloads/AUS_Depression _consumer and carer tx guidelines.pdf', 'file_path': '/Users/misanmeggison/Downloads/AUS_Depression _consumer and carer tx guidelines.pdf', 'page': 0, 'total_pages': 17, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS2 (4.0)', 'producer': 'Adobe PDF Library 7.0', 'creationDate': "D:20090908104936+10'00'", 'modDate': "D:20090908105009+10'00'", 'trapped': ''}),
 Document(page_content='© The Royal Australian and New Zealand College of Psychiatrists, \n2009\nCompiled by The Royal Australian and New Zealand College of Psychiatrists (RANZCP), \nthis information and advice is based on current medical knowledge and practice as at \nthe date of publication. It is intended as a general guide only, and not as a substitute \nfor individual medical advi

In [45]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("/Users/misanmeggison/Downloads/DepressionGuide-web.pdf")

documents = loader.load()

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

# Initialize the embedding model
embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Function to create embeddings
def create_embeddings(text_chunks):
    embeddings = embeddings_model.encode(text_chunks, show_progress_bar=True)
    return embeddings

# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# This text splitter is used to create the child documents
# It should create documents smaller than the parent

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=embeddings_model
)
# The storage layer for the parent documents
store = InMemoryStore()

def rag_retriever(vectorstore, store, parent_splitter, child_splitter):

    retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
    )

    retriever.add_documents(documents)

    return retriever


def retrieve_context(query, top_k):

    # Retrieve the top k similar documents
    sub_docs = vectorstore.similarity_search(query, k=top_k, return_documents=True)

    # Get the context of the first document
    context = sub_docs[0].page_content

    return context

In [46]:
retriever = rag_retriever(vectorstore, store, parent_splitter, child_splitter)

In [48]:
loader = PyMuPDFLoader("/Users/misanmeggison/Downloads/AUS_Depression _consumer and carer tx guidelines.pdf")

docs = loader.load()


retriever.add_documents(docs, ids=None)

len(list(store.yield_keys()))

60

In [49]:
sub_docs = vectorstore.similarity_search("Useful websites", k=8)

sub_docs

[Document(page_content='Organisation\nWebsite\nWhat is the website useful for?\nbeyondblue\nwww.beyondblue.org.au\nInformation on depression, anxiety  \nand bipolar disorder.\nYouth- \nbeyondblue\nwww.youthbeyondblue.com\nbeyondblue’s youth website providing \ninformation on depression and anxiety.\nThe Black Dog \nInstitute\nwww.blackdoginstitute. \norg.au\nSpecialist information on depression and \nbipolar disorder.\nDepression', metadata={'author': '', 'creationDate': "D:20090908104936+10'00'", 'creator': 'Adobe InDesign CS2 (4.0)', 'doc_id': '4e1c981e-7a7d-4f87-81f7-ef9588a36982', 'file_path': '/Users/misanmeggison/Downloads/AUS_Depression _consumer and carer tx guidelines.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20090908105009+10'00'", 'page': 13, 'producer': 'Adobe PDF Library 7.0', 'source': '/Users/misanmeggison/Downloads/AUS_Depression _consumer and carer tx guidelines.pdf', 'subject': '', 'title': '', 'total_pages': 17, 'trapped': ''}),
 Document(page_content=

In [5]:
len(list(store.yield_keys()))


33

In [6]:
sub_docs = vectorstore.similarity_search("anxiety")

print(sub_docs[0].page_content)

attention to any signs of increased anxiety, 
agitation, aggression, or impulsivity. 
Parents should also check their children 
for involuntary restlessness or unexplained 
happiness or energy accompanied by fast, 
driven speech, and unrealistic plans or 
goals. These reactions are more common 
at the start of treatment, but they can 
occur at any time during treatment. If your


In [7]:
retrieved_docs = retriever.get_relevant_documents("depression")

print(type(retrieved_docs[0].page_content))
print(len(retrieved_docs[0].page_content))
print(retrieved_docs[0].page_content)

<class 'str'>
1470
4       Depression: Parents’ Medication Guide
Introduction
T
he original Parents Medical Guide on 
treating depression was published in 
2005, and a revision was published in 
2010, through collaboration by the American 
Academy of Child and Adolescent Psychiatry 
(AACAP) and the American Psychiatric 
Association (APA). The current revision has 
been updated to include new research on 
effective treatments for child and adolescent 
depression. The goal of this guide is to help 
parents make informed decisions about 
getting the best care for a child or adolescent 
with depression.
What is depression?
Depression is a serious illness that can affect 
almost every part of a young person’s life 
and significantly impact his or her family. 
Depression is a type of mood disorder that can 
damage relationships among family members 
and friends, harm school performance, 
and limit other educational opportunities. 
Depression can negatively affect eating, 
sleeping, and physi

In [8]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("nbertagnolli/counsel-chat")
mental_care_df = pd.DataFrame(dataset["train"])

Repo card metadata block was not found. Setting CardData to empty.


In [9]:
# Drop the columns that are not needed
mental_care_df = mental_care_df[['questionTitle', 'answerText']]

In [10]:
# Rename Columns
mental_care_df = mental_care_df.rename(columns={'questionTitle': 'Context', 'answerText': 'Response'})

mental_care_df.shape

(2775, 2)

In [11]:
from datasets import load_dataset

dataset = load_dataset("Amod/mental_health_counseling_conversations")

mental_chat_df = pd.DataFrame(dataset["train"])

In [12]:
mental_chat_df.shape

(3512, 2)

In [13]:
# Append the one dataset to the other
mental_care_df = pd.concat([mental_care_df, mental_chat_df], ignore_index=True)

print(mental_care_df.shape)
mental_care_df.tail()

(6287, 2)


,Context,Response
6282,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
6283,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
6284,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
6285,I think adult life is making him depressed and...,How do you help yourself to believe you requir...
6286,I just took a job that requires me to travel f...,hmm this is a tough one!
